# Entraînement d'un modèle de recommandation

#### Imports

In [2]:
from functions.cleaning import import_collection, clean_whole_df, filter_df, clean_test_df
from functions.database import get_collections_from_db
from functions.evaluation import rate, format_rate_results
from functions.model_processing import partition, run_model

### Connexion à la database et import des données

In [3]:
host = 'localhost'
port = 27017
db_name = 'Movielens'

users, movies = get_collections_from_db(host, port, db_name)
df = import_collection(users)

### Nettoyage du dataframe

In [ ]:
cleaned_df = clean_whole_df(df)

filtering_opts = {
    "min_mean_rating": 1.5,
    "max_mean_rating": 4.5,
    "movies_threshold": 35,
    "movies_few_notes": True,
    "users_threshold": 45,
    "users_few_notes": True,
    "users_no_discriminating": True,
    "users_constant_dt": True,
}

filtered_df = filter_df(cleaned_df, **filtering_opts)

### Partition du dataframe pour l'entraînement du modèle

In [ ]:
partition_opts = {"test_size": 0.8, "mini_size": 0.03}

train_df, test_df, train_mini, test_mini = partition(filtered_df, partition_opts)
train_df.shape, test_df.shape

### Entraînement du modèle

In [ ]:
opts = {
    "n_components": 10,
    "max_iter": 200,
    "normalize": {"should": True, "min": 1, "max": 5},
}

model, predict_matrix = run_model(train_df, opts)
model, predict_matrix.shape

### Nettoyage des données de test

On s'assure ici que le dataframe de test ne comporte uniquement des utilisateurs présents dans le dataframe d'entraînement, puis des films qui ont été évalués par ces mêmes utilisateurs. On effectue ce filtrage afin que l'évaluation du modèle soit cohérente.

In [ ]:
cleaned_test_df = clean_test_df(train_df, test_df)
cleaned_test_df

### Évaluation du modèle

In [ ]:
options = {
    "mse": True,
    "top_10": True,
    "bottom_10": True,
    "ndcg" : True
}

rating_train = rate(predict_matrix, train_df, options)
rating_test = rate(predict_matrix, cleaned_test_df, options)

In [ ]:
results_df = format_rate_results(rating_train, rating_test)
results_df